<a href="https://www.kaggle.com/code/swapnilpanda/test-job?scriptVersionId=113317275" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing libraries for the code

In [13]:
import numpy as np
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten,concatenate 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import pandas as pd

# Giving address of images

In [14]:
train_loc = '/kaggle/input/check3/BF_split/train'
test_loc = '/kaggle/input/check3/BF_split/val'
# test2='pandaimf/swap2/test/'

**Load the train and test data using ImageDataGenerator flow_from_directory. Target size set to 224x224 given that this is the input requirement of VGG16.**

In [15]:
trdata = ImageDataGenerator(rescale=1./255)
traindata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224),batch_size=2)

tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224),batch_size=2)

Found 56 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


**SHowing data classes**

In [16]:
traindata.class_indices

{'Flat foot': 0, 'Normal': 1, 'ToeFirst': 2}

**This part is used for checking**

In [17]:
# from tensorflow.keras.preprocessing import image 

# tdata=image.load_img('/kaggle/input/ta-check/Tibialis Anterior_split/val/flatfoot/19.jpg', target_size=(224,224))
# testd=image.img_to_array(tdata)

# image = np.expand_dims(testd, axis=0)
# best_model.predict(image)

**Use the weights from imagenet to get the pretrained model. The prediction is set to 3 output because we have 3 classes.**

In [18]:
vgg16 = VGG16(weights='imagenet')
vgg16.summary()
x  = vgg16.get_layer('fc2').output
prediction = Dense(3, activation='softmax', name='predictions1')(x)

model = Model(inputs=vgg16.input, outputs=prediction)

2022-12-08 20:22:53.548340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:22:53.641505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:22:53.642242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:22:53.643429: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

553476096/553467096 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

**Now consider that normally, imagenet is recognising objects. We have spectrogram images which has signals. So we set 20 out of 23 layers to trainable.**

# run

# Tibialis Anterior

**** ** All the model parameters ** ****

In [19]:
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[-20:]:
    layer.trainable = True
    print("Layer '%s' is trainable" % layer.name) 
opt = Adam(lr=0.00001)
model.compile(optimizer=opt, loss=categorical_crossentropy, 
              metrics=['accuracy', 'mae'])
model.summary()
checkpoint = ModelCheckpoint("working/vgg16_base_res_QF.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
counter = Counter(traindata.classes)                       
max_val = float(max(counter.values()))   
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
class_weights


Layer 'block1_pool' is trainable
Layer 'block2_conv1' is trainable
Layer 'block2_conv2' is trainable
Layer 'block2_pool' is trainable
Layer 'block3_conv1' is trainable
Layer 'block3_conv2' is trainable
Layer 'block3_conv3' is trainable
Layer 'block3_pool' is trainable
Layer 'block4_conv1' is trainable
Layer 'block4_conv2' is trainable
Layer 'block4_conv3' is trainable
Layer 'block4_pool' is trainable
Layer 'block5_conv1' is trainable
Layer 'block5_conv2' is trainable
Layer 'block5_conv3' is trainable
Layer 'block5_pool' is trainable
Layer 'flatten' is trainable
Layer 'fc1' is trainable
Layer 'fc2' is trainable
Layer 'predictions1' is trainable
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
___

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


{0: 1.0, 1: 1.75, 2: 2.3333333333333335}

# Model training

In [20]:
checkpoint = ModelCheckpoint("working/vgg16_base_res_QF.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')
hist = model.fit(traindata, steps_per_epoch=traindata.samples//traindata.batch_size, validation_data=testdata,
                 class_weight=class_weights,
                   validation_steps=testdata.samples//testdata.batch_size,
                 epochs=150,callbacks=[checkpoint])

2022-12-08 20:23:00.070422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


2022-12-08 20:23:01.731809: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


28/28 [==============================] - 10s 52ms/step - loss: 2.2679 - accuracy: 0.2857 - mae: 0.4495 - val_loss: 1.1542 - val_accuracy: 0.3571 - val_mae: 0.4371

Epoch 00001: val_accuracy improved from -inf to 0.35714, saving model to working/vgg16_base_res_QF.h5
Epoch 2/150
28/28 [==============================] - 1s 39ms/step - loss: 1.8572 - accuracy: 0.3036 - mae: 0.4552 - val_loss: 1.4585 - val_accuracy: 0.2857 - val_mae: 0.4382

Epoch 00002: val_accuracy did not improve from 0.35714
Epoch 3/150
28/28 [==============================] - 1s 39ms/step - loss: 2.0709 - accuracy: 0.3214 - mae: 0.4410 - val_loss: 1.2891 - val_accuracy: 0.2857 - val_mae: 0.4693

Epoch 00003: val_accuracy did not improve from 0.35714
Epoch 4/150
28/28 [==============================] - 1s 39ms/step - loss: 1.7310 - accuracy: 0.4286 - mae: 0.4035 - val_loss: 1.0924 - val_accuracy: 0.2857 - val_mae: 0.4247

Epoch 00004: val_accuracy did not improve from 0.35714
Epoch 5/150
28/28 [=========================

KeyboardInterrupt: 

**Loading model weights**

In [ ]:
from tensorflow.keras.models import load_model
best_model=load_model('/kaggle/working/vgg16_base_res_QF.h5')

**Getting prediction**

In [ ]:
tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224),batch_size=2,shuffle=False)
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = best_model.predict_generator(testdata,testdata.samples//testdata.batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

**Saving results in csv**

In [ ]:
from numpy import savetxt
savetxt('working/Y_pred_BF.csv', Y_pred_BF, delimiter=',')

**CLassification report or result analysis**

In [ ]:
tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224),batch_size=2,shuffle=False)
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = best_model.predict_generator(testdata,testdata.samples//testdata.batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cf_mat=confusion_matrix(testdata.classes, y_pred)
print(confusion_matrix(testdata.classes, y_pred))
print('Classification Report')
target_names = ['Flat Foot','Normal','Toe First']
print(classification_report(testdata.classes, y_pred, target_names=target_names))

Combining results from 4 muscles

In [ ]:
Y_pred_c4 = (Y_pred_Tibialis+Y_pred_QF+Y_pred_GM+y_pred_BF)/4

**Total Results**

In [ ]:

y_pred = np.argmax(Y_pred_QF, axis=1)
print('Confusion Matrix')
cf_mat=confusion_matrix(testdata.classes, y_pred)
print(confusion_matrix(testdata.classes, y_pred))
print('Classification Report')
target_names = ['Flat_foot', 'Normal','ToeFirst']
print(classification_report(testdata.classes, y_pred, target_names=target_names))

# results using colored table

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)
    plt.savefig('working/VGG-imf-ind10.jpg')

In [ ]:
categories = ['Flat Foot','Normal','Toe First']
make_confusion_matrix(cf_mat, categories=categories, figsize=(10,8),cbar=False, title='Confusion matrix of VGG16 from EMG data')

**Training graph**

In [ ]:
# Visualize history
import matplotlib.pyplot as plt
# Plot history: Loss
plt.plot(hist.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.savefig('working/2.jpg')
plt.show()
# Plot history: Accuracy
plt.plot(hist.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.savefig('working/3.jpg')
plt.show()

And lastly we want to visualise how our model is learning. Plot the loss, accuracy, and mae as follows:

In [ ]:
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='val')
plt.title('VGG16: Loss and Validation Loss (0.000001 = Adam LR)')
plt.legend();
plt.savefig('working/4.jpg')
plt.show()

plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='val')
plt.title('VGG16: Accuracy and Validation Accuracy (0.000001 = Adam LR)')
plt.legend();
plt.savefig('working/5.jpg')
plt.show()

plt.plot(hist.history['mae'], label='train')
plt.plot(hist.history['val_mae'], label='val')
plt.title('VGG16: MAE and Validation MAE (0.000001 = Adam LR)')
plt.legend();
plt.savefig('working/6.jpg')
plt.show()

We see that slowly but surely our model is learning our data. Validation accuracy is increasing steadily while mean absolute error is decreasing. However, we should be aware that loss is increasing. In this context, but maybe we can just ditch loss since we are already looking at mae.

And that's that! If you're following this series, thank you so much! Please upvote if this helped you in any way.
